In [1]:
library(tidyverse)
library(lubridate)
options(digits = 22)

── Attaching packages ─────────────────────────────────────── tidyverse 1.2.1 ──

✔ ggplot2 3.2.1     ✔ purrr   0.3.2
✔ tibble  2.1.3     ✔ dplyr   0.8.3
✔ tidyr   1.0.0     ✔ stringr 1.4.0
✔ readr   1.3.1     ✔ forcats 0.4.0

── Conflicts ────────────────────────────────────────── tidyverse_conflicts() ──
✖ dplyr::filter() masks stats::filter()
✖ dplyr::lag()    masks stats::lag()


Attaching package: ‘lubridate’


The following object is masked from ‘package:base’:

    date




In [2]:
crime = read_csv('/Users/wenjiazhai/Documents/GitHub/python/TDI_2020/Arrest_Data_from_2010_to_Present.csv')
head(crime)

Parsed with column specification:
cols(
  `Report ID` = col_double(),
  `Arrest Date` = col_character(),
  Time = col_character(),
  `Area ID` = col_character(),
  `Area Name` = col_character(),
  `Reporting District` = col_character(),
  Age = col_double(),
  `Sex Code` = col_character(),
  `Descent Code` = col_character(),
  `Charge Group Code` = col_character(),
  `Charge Group Description` = col_character(),
  `Arrest Type Code` = col_character(),
  Charge = col_character(),
  `Charge Description` = col_character(),
  Address = col_character(),
  `Cross Street` = col_character(),
  Location = col_character()
)



Report ID,Arrest Date,Time,Area ID,Area Name,Reporting District,Age,Sex Code,Descent Code,Charge Group Code,Charge Group Description,Arrest Type Code,Charge,Charge Description,Address,Cross Street,Location
<dbl>,<chr>,<chr>,<chr>,<chr>,<chr>,<dbl>,<chr>,<chr>,<chr>,<chr>,<chr>,<chr>,<chr>,<chr>,<chr>,<chr>
4248313,02/24/2015,1310,20,Olympic,2022,37,M,H,05,Burglary,F,459PC,BURGLARY,5TH,WILTON,"(34.0653, -118.314)"
191811472,05/03/2019,1700,18,Southeast,1802,23,F,B,NA,NA,M,653.22 PC,NA,91ST,FIGUEROA,"(33.9543, -118.2827)"
4254777,02/26/2015,2010,19,Mission,1985,22,M,H,06,Larceny,M,459.5PC,SHOPLIFTING,8300 VAN NUYS BL,NA,"(34.2216, -118.4488)"
5614161,04/29/2019,1040,08,West LA,0842,41,M,H,03,Robbery,F,211PC,ROBBERY,11600 WILSHIRE BL,NA,"(34.0508, -118.4592)"
5615197,04/30/2019,0615,06,Hollywood,0663,27,M,O,05,Burglary,F,459PC,BURGLARY,LA BREA,LEXINGTON,"(34.0907, -118.3384)"
5615701,04/30/2019,1100,09,Van Nuys,0901,2,F,H,NA,NA,D,300(B)WIC,NA,RAYMER,SEPULVEDA BL,"(34.2149, -118.4674)"


# - How many bookings of arrestees were made in 2018?

In [3]:
crime$`Arrest Date` <- as.Date(crime$`Arrest Date`, format='%m/%d/%Y')

crime_2018 <- crime[crime$`Arrest Date` >= as.Date('01/01/2018', format='%m/%d/%Y') & crime$`Arrest Date` <= as.Date('12/31/2018', format='%m/%d/%Y'),]

nrow(crime_2018)

[1] 104277

# - How many bookings of arrestees were made in the area with the most arrests in 2018?

In [4]:
arrests <- crime_2018 %>%
    group_by(`Area Name`) %>%
    count() %>%
    arrange(desc(n))

arrests$n[1]

[1] 10951

# - What is the 95% quantile of the age of the arrestee in 2018? Only consider the following charge groups for your analysis: Vehicle Theft, Robbery, Burglary, Receive Stolen Property.

In [5]:
crime_specific <- crime_2018 %>%
    filter(`Charge Group Description` %in% c('Vehicle Theft', 'Robbery', 'Burglary', 'Receive Stolen Property'))

quantile(crime_specific$Age, 0.95)

95% 
 52

# - There are differences between the average age of an arrestee for the various charge groups. Are these differences statistically significant? For this question, calculate the Z-score of the average age for each charge group. Report the largest absolute value among the calculated Z-scores.

# Only consider data for 2018. Do not consider "Pre-Delinquency" and "Non-Criminal Detention" as these charge groups are reserved for minors. Exclude any arrests where the charge group description is not known.

In [6]:
avg_age <-  crime_2018 %>%
    group_by(`Charge Group Description`) %>%
    summarise(avg_age = mean(Age))

avg = mean(avg_age$avg_age)
sd = sd(avg_age$avg_age)
n = length(avg_age$avg_age)

t <- (mean(avg_age$avg_age) - 0.7)/(sd(avg_age$avg_age) / sqrt(length(avg_age$avg_age)))
2*pt(-abs(t),df=length(avg_age$avg_age)-1)

[1] 7.72283487914759568357e-20

### The p-value is smaller than 0.05, so the difference is significant.

In [7]:
max(abs(scale(avg_age$avg_age)))

[1] 3.319857461037262424952

# - Felony arrest incidents have been dropping over the years. Using a trend line (linear estimation) for the data from 2010 and 2018 (inclusive), what is the projected number of felony arrests in 2019?

In [8]:
sum(grepl('Felony', unique(crime_2018$`Charge Group Description`), ignore.case = TRUE))
sum(grepl('Felony', unique(crime_2018$`Charge Description`), ignore.case = TRUE))

[1] 0

[1] 8

In [9]:
felony <- crime %>%
    mutate(year = year(`Arrest Date`)) %>%
    filter(year %in% c(2010:2018), grepl('Felony', `Charge Description`, ignore.case = TRUE)) %>%
    group_by(year) %>%
    summarise(n = n())

lm(n ~ year, felony)


Call:
lm(formula = n ~ year, data = felony)

Coefficients:
            (Intercept)                     year  
862728.3999999999068677    -427.4333333333332803  


In [10]:
round(862728.4 + (-427.4) * 2019)

[1] -192

# - Some types of arrest incidents in certain areas occur at a highly disproportionate rate compared to their frequency city-wide. For example, let's say that the rate of larceny arrests (charge group code 6) is 1% in Devonshire (area ID 17). This rate may appear low but what if larceny arrests constitute 0.1 % city-wide? The ratio between these two probabilities is 10 and we can say that larceny occurs unusually often in Devonshire (Note, these numbers were made up for illustration purposes). Calculate this ratio for all charge group code and area ID pairs. You can view this ratio as the ratio of the conditional probability of an arrest incident of a charge group code given that it occurred in an area ID to the unconditional probability of the arrest incident of a charge group. Report the average of the top 5 of the calculated ratio.

# Consider all records prior to January 1, 2019. Some arrest incidents don't have a charge group code. These records should not be considered in your analysis. Arrest incidents for charge group code 99 should not be considered in your analysis.

In [11]:
crime_per_area <- crime %>%
    mutate(year = year(`Arrest Date`)) %>%
    filter(year < 2019, !(`Charge Group Code` %in% c(99, NA))) %>%
    group_by(`Area ID`, `Charge Group Code`) %>%
    summarise(n = n()) %>%
    pivot_wider(names_from = `Charge Group Code`, values_from = n, names_prefix = 'code_') %>%
    ungroup() %>%
    select(-`Area ID`)


prop_crime_rate <- prop.table(as.matrix(crime_per_area), 2)
mean(sort(apply(prop_crime_rate, 1, max), decreasing=TRUE)[0:5])

[1] 0.2479092263354117364749

# - How many arrest incidents occurred within 2 km from the Bradbury Building in 2018? Use (34.050536, -118.247861) for the coordinates of the Bradbury Building . For simplicity, please use the spherical Earth projected to a plane equation for calculating distances. Use the radius of the Earth as 6371 km. Note, some arrest records are missing location data and the location is listed as (0, 0). These records should not factor in your calculation.

In [12]:
crime_geo <- crime_2018 %>%
    filter(Location != '(0, 0)') %>%
    mutate(year = year(`Arrest Date`)) %>%
#     filter(grepl('Pico', Address, ignore.case = TRUE)) %>%
    extract(Location, c("Lat", "Lon"), "([\\d.]+)[^\\d.]+(-[\\d.]+)", convert = TRUE) %>%
    mutate(Lat = as.numeric(Lat), Lon = as.numeric(Lon))

head(crime_geo)

Report ID,Arrest Date,Time,Area ID,Area Name,Reporting District,Age,Sex Code,Descent Code,Charge Group Code,Charge Group Description,Arrest Type Code,Charge,Charge Description,Address,Cross Street,Lat,Lon,year
<dbl>,<date>,<chr>,<chr>,<chr>,<chr>,<dbl>,<chr>,<chr>,<chr>,<chr>,<chr>,<chr>,<chr>,<chr>,<chr>,<dbl>,<dbl>,<dbl>
5213460,2018-01-24,1930,06,Hollywood,0668,29,M,H,10,Fraud/Embezzlement,F,530.5(A)PC,GET CREDIT/ETC OTHER'S ID,ST ANDREWS,FOUNTAIN,34.09490000000000264890,-118.3109000000000037289,2018
5257944,2018-03-15,0310,07,Wilshire,0743,33,F,O,01,Homicide,F,191.5(B)PC,VEH MANSLAUGHTR/OPER VESSEL DUI & DUR FEL,OLYMPIC,MASSELIN,34.05740000000000122782,-118.3546999999999940201,2018
5328813,2018-06-01,1800,03,Southwest,0356,44,F,B,16,Narcotic Drug Laws,F,11379(A)HS,TRANSPORT/SELL CONTROLLED SUBSTANCE,1600 W 36TH PL,NA,34.02190000000000225100,-118.3061000000000007049,2018
5336331,2018-06-10,0200,13,Newton,1364,23,M,H,12,Weapon (carry/poss),F,25400(A)2PC,CARRYING CONCEALED WEAPON UPON PERSON,1200 E 50TH ST,NA,33.99799999999999755573,-118.2543000000000006366,2018
5360731,2018-07-07,2330,18,Southeast,1832,32,M,B,12,Weapon (carry/poss),F,29800(A)1PC,POSS F/ARM BY CONVICTED FELON/ADDICT/ETC,10700 S MAIN ST,NA,33.93919999999999959073,-118.2738999999999975898,2018
5404690,2018-08-25,1850,18,Southeast,1832,39,M,B,12,Weapon (carry/poss),F,29800(A)1PC,POSS F/ARM BY CONVICTED FELON/ADDICT/ETC,10600 S MAIN ST,NA,33.94019999999999726015,-118.2738999999999975898,2018


In [13]:
crime_geo %>%
    mutate(distance = 6371 * sqrt(((Lat - 34.050536) * pi / 180) ^ 2 + cos(Lon) * ((Lon - (-118.247861)) * pi / 180) ^ 2)) %>%
    filter(distance < 2) %>%
    summarise(n())

n()
<int>
12421


# - How many arrest incidents were made per kilometer on Pico Boulevard during 2018? For this question, we will need to estimate the length of Pico Boulevard, which mostly stretches from east to west. To estimate the length of Pico Boulevard:

# Consider all location data which the listed address mentions "Pico". Remove outliers by filtering out locations where either the latitude or longitude is 2 standard deviations beyond the mean of the subset of identified points. To estimate the length, calculate the distance from the most western and eastern coordinate points. As before, use the simplified flat surface equation.

# Once you have estimated the length of Pico Boulevard, you can proceed to report the number of arrest incidents per kilometer on Pico Boulevard in 2018.

In [14]:
pico <- crime_2018 %>%
    filter(Location != '(0, 0)') %>%
    mutate(year = year(`Arrest Date`)) %>%
    filter(grepl('Pico', Address, ignore.case = TRUE)) %>%
    extract(Location, c("Lat", "Lon"), "([\\d.]+)[^\\d.]+(-[\\d.]+)", convert = TRUE) %>%
    mutate(Lat = as.numeric(Lat), Lon = as.numeric(Lon))

In [15]:
pico <- pico %>%
    filter(Lat - mean(Lat) < 2 * sd(Lat), Lon - mean(Lon) < 2 * sd(Lon))

num <- nrow(pico)


num / (6371 * abs(max(pico$Lon) - min(pico$Lon)) * pi / 180)

[1] 23.02354343284158133542